In [ ]:
import torch
from occhio import ToyModel
from occhio.model_grid import ModelGrid, Axis
from occhio.autoencoder import TiedLinearRelu
from occhio.distributions import CorrelatedPairs, HierarchicalPairs
from occhio.visualization.phase_change import plot_phase_change
from occhio.visualization.export import export_figure

In [ ]:
generator = torch.Generator("cpu").manual_seed(42)

In [ ]:
N_FEATURES = 2
N_HIDDEN = 1
P_INDIVIDUAL = 1
P_FOLLOW = 1
DATA = "correlated_pairs"
EXPERIMENT_SIZE = 12

In [ ]:
densities = torch.logspace(0, -2, EXPERIMENT_SIZE)
importances = torch.logspace(-1, 1, EXPERIMENT_SIZE)

In [ ]:
def model_trainer(density, importance):
    model = ToyModel(
        distribution=CorrelatedPairs(
            N_FEATURES, p_active=density, p_individual=P_INDIVIDUAL, generator=generator
        ),
        ae=TiedLinearRelu(N_FEATURES, N_HIDDEN, generator=generator),
        importances=importance ** torch.arange(N_FEATURES),
    )
    model.fit(n_epochs=2000)
    return model


model_grid = ModelGrid(
    model_trainer,
    x_axis=Axis("Importance Decay", importances),
    y_axis=Axis("Density", densities),
)

In [ ]:
print(model_grid.models[0][0].ae.W.shape)
print(model_grid.models[0][0].W.shape)

In [ ]:
fig = plot_phase_change(model_grid, tracked_feature=0)
fig.show()

In [ ]:
export_figure(
    fig,
    {"data": DATA, "p_individual": P_INDIVIDUAL},
    # {"data": DATA, "p_follow": P_FOLLOW},
    subdir="phase-changes",
)